In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
def convblock(in_channels, out_channels,kernel_size=3,stride=1,padding=1,size=2):
    conv_layers = nn.ModuleList()
    for i in range(size):
        conv_layers.append(nn.Conv2d(in_channels, out_channels,kernel_size=3,stride=1,padding=1))
        conv_layers.append(nn.ReLU())
        in_channels = out_channels
    conv_layers.append(nn.MaxPool2d(kernel_size=2,stride=2))
    return conv_layers

In [3]:
def linearblock(in_features, out_features,size=2):
    linear_layers = nn.ModuleList()
    for i in range(size-1):
        linear_layers.append(nn.Linear(in_features,out_features))
        linear_layers.append(nn.Dropout(p=0.5))
        in_features = out_features
    linear_layers.append(nn.Linear(out_features, 1000))
    linear_layers.append(nn.Softmax(dim=1))
    return linear_layers
        

In [4]:
class vgg16(nn.Module):
    def __init__(self):
        super(vgg16, self).__init__()
        self.block1 = convblock(3,64,size=2) # First Block
        self.block2 = convblock(64,128,size=2) # Second Block
        self.block3 = convblock(128,256,size=3) # Third Block
        self.block4 = convblock(256,512,size=3) # Fourth Block
        self.block5 = convblock(512,512,size=3) # Fifth Block
        self.pool = nn.AdaptiveAvgPool2d((7,7))
        self.classifier = linearblock(7*7*512,4096,size=3)

    def forward(self, x):
        blocks = [self.block1, self.block2, self.block3, self.block4, self.block5]
        for block in blocks:
            for layer in block:
                x = layer(x)
        x = self.pool(x)
        x = x.view(-1, 7*7*512)
        for layer in self.classifier:
            x = layer(x)
        return x



In [5]:
model_vgg = vgg16()

In [6]:
x = torch.randn(4,3,224,224)
output = model_vgg(x)
output.shape

torch.Size([4, 1000])

In [7]:
class vgg19(nn.Module):
    def __init__(self):
        super(vgg19, self).__init__()
        self.block1 = convblock(3,64,size=2) # First Block
        self.block2 = convblock(64,128,size=2) # Second Block
        self.block3 = convblock(128,256,size=4) # Third Block
        self.block4 = convblock(256,512,size=4) # Fourth Block
        self.block5 = convblock(512,512,size=4) # Fifth Block
        self.pool = nn.AdaptiveAvgPool2d((7,7))
        self.classifier = linearblock(7*7*512,4096,size=3)

    def forward(self, x):
        blocks = [self.block1, self.block2, self.block3, self.block4, self.block5]
        for block in blocks:
            for layer in block:
                x = layer(x)
        x = self.pool(x)
        x = x.view(-1, 7*7*512)
        for layer in self.classifier:
            x = layer(x)
        return x



In [8]:
model_vgg19 = vgg19()

In [9]:
x = torch.randn(4,3,224,224)
output = model_vgg19(x)
output.shape

torch.Size([4, 1000])